In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [15]:
# hyperparameters
N_batch = 12
N_block = 64
max_iters = 2000
eval_interval = 20
lr = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 20
N_embd = 128
N_head = 4
N_layer = 4
dropout = 0.0

In [16]:
torch.manual_seed(1337)

In [17]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [18]:
chars = sorted(list(set(text)))
N_vocab = len(chars)
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s:[stoi[c] for c in s]
decode = lambda l:''.join([itos[i] for i in l])

In [19]:
data = torch.tensor(encode(text), dtype = torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
valid_data = data[n:]

In [20]:
def get_batch(split):
    data = train_data if split == 'train' else valid_data
    ix = torch.randint(len(data) - N_block, (N_batch,))
    x = torch.stack([data[i:i+N_block] for i in ix])
    y = torch.stack([data[i+1:i+N_block+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y


In [21]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','valid']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [22]:
class Head(nn.Module):

    def __init__(self, N_head):
        super().__init__()
        self.key = nn.Linear(N_embd, N_head, bias = False)
        self.query = nn.Linear(N_embd, N_head, bias = False)
        self.value = nn.Linear(N_embd, N_head, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(N_block, N_block)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        
        wt = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wt = wt.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wt = F.softmax(wt, dim = -1)
        wt = self.dropout(wt)
        v = self.value(x)
        out = wt @ v
        return out

In [23]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, N_head):
        super().__init__()
        self.heads = nn.ModuleList([Head(N_head) for _ in range(num_heads)])
        self.proj = nn.Linear(N_head * num_heads, N_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.dropout(self.proj(out))
        return out

In [24]:
class FeedForward(nn.Module):

    def __init__(self, N_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(N_embd, 4*N_embd),
            nn.ReLU(),
            nn.Linear(4*N_embd, N_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [25]:
class Block(nn.Module):

    def __init__(self, N_embd, N_head):
        super().__init__()
        head_size = N_embd // N_head
        self.sa = MultiHeadAttention(N_head, head_size)
        self.ffwd = FeedForward(N_embd)
        self.ln1 = nn.LayerNorm(N_embd)
        self.ln2 = nn.LayerNorm(N_embd)

    def forward(self,x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [26]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(N_vocab, N_embd)
        self.position_embedding_table = nn.Embedding(N_block, N_embd)
        self.blocks = nn.Sequential(*[Block(N_embd, N_head = N_head) for _ in range(N_layer)])
        self.ln_f = nn.LayerNorm(N_embd)
        self.lm_head = nn.Linear(N_embd, N_vocab)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)

    def forward(self, idx, targets = None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -N_block:]
            logits, loss = self(idx_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)

        return idx

In [27]:
model = GPTLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.816705 M parameters


In [28]:
optimizer = torch.optim.AdamW(model.parameters(), lr = lr)

In [30]:
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss{losses['valid']:.4f}")

    xb,yb = get_batch('train')

    logits, loss = model(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.1906, val loss4.1877
step 20: train loss 3.2445, val loss3.2831
step 40: train loss 2.8555, val loss2.9049
step 60: train loss 2.6872, val loss2.6790
step 80: train loss 2.5975, val loss2.6268
step 100: train loss 2.5637, val loss2.5715
step 120: train loss 2.5440, val loss2.5218
step 140: train loss 2.5250, val loss2.5188
step 160: train loss 2.5064, val loss2.4672
step 180: train loss 2.4957, val loss2.4844
step 200: train loss 2.4632, val loss2.4795
step 220: train loss 2.4407, val loss2.4453
step 240: train loss 2.4357, val loss2.4347
step 260: train loss 2.3990, val loss2.4146
step 280: train loss 2.3802, val loss2.3861
step 300: train loss 2.3599, val loss2.3624
step 320: train loss 2.3446, val loss2.3530
step 340: train loss 2.3401, val loss2.3542
step 360: train loss 2.2993, val loss2.3242
step 380: train loss 2.3189, val loss2.2969
step 400: train loss 2.2594, val loss2.2842
step 420: train loss 2.2537, val loss2.2687
step 440: train loss 2.2376, val loss2

In [31]:
context = torch.zeros((1,1), dtype = torch.long, device = device)
print(decode(m.generate(context, max_new_tokens = 500)[0].tolist()))


I'll think acce: on thy thas sceeven maday
Till feaping so varvingse, haves pots,
Penow to rid, I love can hit well were with dad,
Who? The poestely of in my kingge, Andil a shame,
That have his see romms to lopre go men?

DUKE VINCENTIO:
Done, goe, inselconess, bother, fork over praya,
Slanglal repon the insen,
Buning out lords these her faway, so mose,
Tuslullains myse mose fast--be
we a caurt throu no like appeop on to were
That whe whas wise for dect, I dearing as I all kist
God causpy speec
